In [1]:
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1012. Allostatic Load/P1012.3. Analyses/P1012.3.2. Ordinal Regression/Output')
cur_date = "060124"

library(readxl)
library(openxlsx)
library(tidyverse)
library(ordinalNet)
library(reshape2)

# reading in file
#biomarker_df = data.frame(read_excel("Input/Allostatic_Mediator_Data_050824.xlsx", sheet = 2)) # GO BACK FOR IMPUTED VALUES!!!
bp_df = data.frame(read_excel("Input/Allostatic_Mediator_Data_050824.xlsx", sheet = 4)) 
subject_info_df = data.frame(read_excel("Input/Subject_Info_050824.xlsx", sheet = 2))
mediator_scores_df = data.frame(read_excel("Input/Mediator_Scores_BP_Subjects_052724.xlsx"))

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths




In [2]:
head(bp_df)
head(subject_info_df)
head(mediator_scores_df)

,Subject_ID,Systolic,Diastolic,BP_Classification
,<chr>,<dbl>,<dbl>,<chr>
1,CS_26,133,86,1
2,CS_29,116,84,N
3,CS_17,118,80,N
4,NS_63,122,85,E
5,CS_21,112,77,N
6,CS_27,131,91,1


,Study,Original_Subject_Number,Subject_Number,Group,Subject_ID,Sex,Age,Race
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>
1,FM2,97,1,CS,CS_1,M,18,B
2,FM2,99,2,CS,CS_2,F,20,W
3,FM2,101,3,CS,CS_3,F,20,W
4,FM2,98,4,CS,CS_4,F,21,W
5,TCORS LAIV,39,5,CS,CS_5,M,21,W
6,FM2,100,6,CS,CS_6,F,22,W


,Variable,Subject_ID,Mediator_Score
,<chr>,<chr>,<dbl>
1,Cortisol,CS_5,0.24743096
2,Cortisol,CS_8,0.31083055
3,Cortisol,CS_10,0.13144609
4,Cortisol,CS_16,0.06754252
5,Cortisol,CS_17,0.44649218
6,Cortisol,CS_21,0.21687581


In this analysis, we're using [ordinal regression](https://www.statisticssolutions.com/free-resources/directory-of-statistical-analyses/ordinal-regression/) to predict blood pressure classification using mediator scores from 18 biomarkers. 

In [3]:
# creating a vector for variables that increase atherosclerotic risk
# HDL decreases risk
allostatic_load_biomarkers = c('Cortisol','Noradrenaline','Epinephrine','Hba1c','Fibrinogen','CRP', 'HDL')

# creating 1 df
ordinal_regression_df = inner_join(bp_df[,c(1,4)], subject_info_df[,4:8]) %>%
    # adding allostatic load data
    inner_join(mediator_scores_df) %>%
    # obtaining allostatic load biomarkers only
    #filter(Variable %in% allostatic_load_biomarkers) %>%
    pivot_wider(names_from = Variable, values_from = Mediator_Score)

# turning BP into a factor
ordinal_regression_df$BP_Classification = factor(ordinal_regression_df$BP_Classification, levels = c("N", "E", "1", '2'))

#reordering columns
ordinal_regression_df = ordinal_regression_df[,c(1,3:22,24,23,2)]
    

head(ordinal_regression_df)

Joining with `by = join_by(Subject_ID)`
Joining with `by = join_by(Subject_ID)`


Subject_ID,Group,Sex,Age,Race,Cortisol,Noradrenaline,Hba1c,Fibrinogen,CRP,⋯,IL13,IL1B,IL2,IL4,IL6,IL8,TNFa,Epinephrine,HDL,BP_Classification
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
CS_26,CS,F,34,B,0.15123818,0.00000000,0.1261076,0.35238170,0.03189947,⋯,0.3271404,0.00000000,2.351685e-07,0.25823172,0.18675526,0.001686759,0.14780720,0.09189689,0.01961938,1
CS_29,CS,F,36,B,0.06791410,0.09028609,0.3276962,0.18618128,0.04754286,⋯,0.7220417,0.01153174,1.419332e-08,0.53046857,0.43704343,0.355268670,0.08243793,0.48260060,0.29810436,N
CS_17,CS,M,29,W,0.44649218,0.32976117,0.2873836,0.18107212,0.01664812,⋯,0.3411684,0.00000000,6.761111e-08,0.49508420,0.03189408,0.147350214,0.26384236,0.72404411,0.52779975,N
NS_63,NS,M,43,B,0.21687581,0.25782803,0.4462655,0.11215630,0.22912585,⋯,0.6317240,0.00000000,1.132208e-06,0.60115086,1.00000000,0.119470619,0.42067290,0.67048546,0.13049306,E
CS_21,CS,F,31,B,0.21687581,0.27113435,0.7550424,0.16931582,0.12987698,⋯,0.2035670,0.00000000,9.701944e-07,0.07874693,0.30264289,0.418614119,0.26384236,0.31548045,0.06673031,N
CS_27,CS,F,35,W,0.09156722,0.23404945,0.3807725,0.03079665,0.02882542,⋯,0.4987997,0.00000000,7.491238e-08,0.35320369,0.14274251,0.033546943,0.38209701,0.00000000,0.14858426,1


In [4]:
# creating a vector of booleans that specifies whether or not the coefficient should be
# positive (TRUE) or negative (FALSE), only HDL will be negative
biomarker_coefficients = c(rep(TRUE, ncol(ordinal_regression_df) - 7), FALSE)

estimateOrdWeights <- function(dataset, response, coefficients_magnitude){
    # """
    # Creating a function to run ordinal regression to estimate ordinal weights of mediator data based
    # on blood pressure classification
    # :param (input): wide df with biomarker data, response variable, magnitude of the coefficients
    # :output: 1 df with the ordinal weights
    # """
    
    #get number of response levels
    NoBins <- length(unique(dataset[[response]]))

    #get number of features or slieces
    noSlices <- ncol(dataset) - 2
    
    # obtaining the features and converting into a matrix
    feature_data <- as.matrix(dataset[,c(2:(ncol(dataset) - 1))])

    # ordinal regression
    # positiveID specifies whether or not each coefficient for each ind. variable should be constrained to be non-negative
    regression_model <- ordinalNet(feature_data, dataset[[response]], positiveID = coefficients_magnitude, 
                            # lambdaVals defines how quickly your coefficients drop to 0
                            # keeping it at 0 tries to minimize feature reduction
                            # maxiterout defines the number of iterations before it stops
                         lambdaVals = 0, maxiterOut = 100)
    testW <- regression_model$coefs[NoBins:(noSlices + NoBins - 1)]

    # rescaling the coefficients so that they sum to 1
    normalizer = 1/sum(testW)
    rescaled_weight <- testW * normalizer

    # adding the variable names back in 
    test_weights_df = data.frame(Variable = colnames(feature_data), Coefficient = rescaled_weight) %>%
        arrange(-Coefficient) 
    
    return(test_weights_df)
}

# calling fn
biomarker_coefficients_df = estimateOrdWeights(ordinal_regression_df[,c(1,6:24)], 'BP_Classification', biomarker_coefficients)

head(biomarker_coefficients_df)

,Variable,Coefficient
,<chr>,<dbl>
1,IL2,0.32461122
2,IL8,0.17739730
3,IL12p70,0.17591443
4,HDL,0.08853286
5,IL10,0.07997711
6,Epinephrine,0.04285506


In [5]:
biomarker_coefficients_df

Variable,Coefficient
<chr>,<dbl>
IL2,0.32461122
IL8,0.17739730
IL12p70,0.17591443
HDL,0.08853286
IL10,0.07997711
Epinephrine,0.04285506
Cortisol,0.04247511
IL13,0.02969636
IL4,0.01933184


Why wasn't HDL able to be constrained to negative?

Based on these results, we can see that IL-2 had the largest (36%) contribution to blood pressure classification.

# Allostatic Load Calculation using Linear Regression

Now that we have these weights from the ordinal regression, we'll use a linear regression to calculate allostatic load scores for each subject using both the previously calculated mediator scores multiplied the ordinal weight for the 7 allostatic load biomarkers (cortisol, noradrenaline, epinephrine, HDL, Hba1c, fibrinogen, and CRP). The formula looks like...

Allostatic Load = Cortisol ordinal weight * Cortisol score + CRP ordinal weight * CRP score...

First, we'll combine the mediator scores with the ordinal weights into one dataframe.

In [6]:
wider_scores_df = dcast(mediator_scores_df, Variable ~ Subject_ID, value.var = "Mediator_Score") %>% 
    inner_join(biomarker_coefficients_df) %>%
    # obtaining allostatic load biomarkers only
    filter(Variable %in% allostatic_load_biomarkers) %>%
    column_to_rownames("Variable")

wider_scores_df

Joining with `by = join_by(Variable)`


,CS_10,CS_16,CS_17,CS_21,CS_22,CS_23,CS_24,CS_26,CS_27,CS_28,⋯,NS_54,NS_56,NS_57,NS_58,NS_59,NS_60,NS_61,NS_62,NS_63,Coefficient
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Cortisol,0.13144609,0.06754252,0.44649218,0.21687581,0.04373137,0.03011523,0.22022432,0.15123818,0.09156722,0.00000000,⋯,0.162261782,0.21854579,0.24564138,0.1446778341,0.1936156,0.15076409,0.2174310,0.05219233,0.2168758,0.04247511
CRP,0.06081895,0.02650878,0.01664812,0.12987698,0.03117062,0.03881788,0.09376314,0.03189947,0.02882542,0.03715937,⋯,0.003752825,0.14078284,0.09701449,0.0002840778,0.6071295,0.33981318,0.3777298,0.98635392,0.2291258,0.00000000
Epinephrine,0.36245167,1.00000000,0.72404411,0.31548045,0.64614063,0.07919232,0.14750107,0.09189689,0.00000000,0.50379493,⋯,0.095549052,0.09458031,0.75869970,0.2421595303,0.3192038,0.09565971,0.5938708,0.26922526,0.6704855,0.04285506
Fibrinogen,0.26137633,0.00000000,0.18107212,0.16931582,0.10842287,0.44374148,0.27818307,0.35238170,0.03079665,0.11464564,⋯,0.100644949,0.21754258,0.41187896,0.1571598428,0.2116680,0.16188742,0.6245899,0.30306729,0.1121563,0.00000000
Hba1c,0.64414592,0.11121726,0.28738359,0.75504241,0.57615251,0.73684025,0.78829727,0.12610762,0.38077250,0.20661601,⋯,0.296014746,0.33486693,0.41211041,0.2201362250,0.7033799,0.15901222,0.0000000,0.67872332,0.4462655,0.00000000
HDL,0.02491066,0.17484827,0.52779975,0.06673031,0.14420877,0.26613495,0.24611656,0.01961938,0.14858426,0.07812076,⋯,0.243013639,0.09035361,0.28415615,0.2342484850,0.1731317,0.20476993,0.5024475,0.13228858,0.1304931,0.08853286
Noradrenaline,0.84665641,0.50181436,0.32976117,0.27113435,0.33094643,0.22862238,0.67059913,0.00000000,0.23404945,0.08305481,⋯,0.333338197,0.17657712,0.08418630,0.2720013321,0.1512783,0.22996146,0.6586311,0.47647309,0.2578280,0.01920870


In [7]:
# multiplying the value of the mediator in each row by its respective ordinal
# weight found in the last column
multiplication_df = wider_scores_df %>%
    select(-Coefficient) %>%
    mutate(across(contains("_"), ~.x*wider_scores_df$Coefficient))

# # identifying the rows with only zeros 
# zero_booleans = apply(multiplication_df, 1, function(row) all(row == 0)) 
# zero_rownames = rownames(multiplication_df)[zero_booleans]

# `%notin%` <- Negate(`%in%`)
# zero_df = wider_biomarker_df[rownames(wider_biomarker_df) %in% zero_rownames,] %>%
#     select(-Coefficient)
# non_zero_df = multiplication_df[rownames(multiplication_df) %notin% zero_rownames,] 
                          
# # recombining data
# weighted_mediator_score_df = rbind(zero_df, non_zero_df)
                      
# head(weighted_mediator_score_df)

multiplication_df

,CS_10,CS_16,CS_17,CS_21,CS_22,CS_23,CS_24,CS_26,CS_27,CS_28,⋯,NS_53,NS_54,NS_56,NS_57,NS_58,NS_59,NS_60,NS_61,NS_62,NS_63
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Cortisol,0.005583187,0.002868876,0.018964803,0.009211823,0.001857494,0.001279148,0.009354051,0.006423858,0.003889328,0.000000000,⋯,1.421884e-02,0.006892086,0.009282756,0.01043364,0.006145206,0.008223844,0.006403721,0.009235407,0.002216875,0.009211823
CRP,0.000000000,0.000000000,0.000000000,0.000000000,0.000000000,0.000000000,0.000000000,0.000000000,0.000000000,0.000000000,⋯,0.000000e+00,0.000000000,0.000000000,0.00000000,0.000000000,0.000000000,0.000000000,0.000000000,0.000000000,0.000000000
Epinephrine,0.015532887,0.042855058,0.031028952,0.013519933,0.027690394,0.003393792,0.006321167,0.003938247,0.000000000,0.021590161,⋯,2.351106e-02,0.004094760,0.004053245,0.03251412,0.010377761,0.013679497,0.004099502,0.025450369,0.011537664,0.028733693
Fibrinogen,0.000000000,0.000000000,0.000000000,0.000000000,0.000000000,0.000000000,0.000000000,0.000000000,0.000000000,0.000000000,⋯,0.000000e+00,0.000000000,0.000000000,0.00000000,0.000000000,0.000000000,0.000000000,0.000000000,0.000000000,0.000000000
Hba1c,0.000000000,0.000000000,0.000000000,0.000000000,0.000000000,0.000000000,0.000000000,0.000000000,0.000000000,0.000000000,⋯,0.000000e+00,0.000000000,0.000000000,0.00000000,0.000000000,0.000000000,0.000000000,0.000000000,0.000000000,0.000000000
HDL,0.002205412,0.015479817,0.046727621,0.005907825,0.012767215,0.023561688,0.021789403,0.001736960,0.013154589,0.006916254,⋯,4.546348e-02,0.021514693,0.007999264,0.02515716,0.020738688,0.015327843,0.018128867,0.044483117,0.011711887,0.011552924
Noradrenaline,0.016263172,0.009639203,0.006334285,0.005208139,0.006357052,0.004391540,0.012881340,0.000000000,0.004495786,0.001595375,⋯,6.646191e-05,0.006402995,0.003391818,0.00161711,0.005224793,0.002905861,0.004417262,0.012651450,0.009152430,0.004952542


In [8]:
# calculating allostatic load by summing by the columns
allostatic_load_df = data.frame(Allostatic_Load = apply(multiplication_df, 2, sum)) %>%
    arrange(-Allostatic_Load)

head(allostatic_load_df)

,Allostatic_Load
,<dbl>
NS_41,0.11948496
CS_31,0.11776398
CS_17,0.10305566
NS_38,0.09541933
NS_61,0.09182034
NS_53,0.08325984
